# init video editing studio

In [4]:
import psycopg2
from pytube import YouTube
import os
import moviepy.editor as mpy
from dotenv import load_dotenv
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip

# Load environment variables from .env file
load_dotenv()

# Step 1: Fetch YouTube links from the database
def get_youtube_links():
    DBHOST = os.getenv('DBHOST')
    DBPORT = os.getenv('DBPORT')
    DBNAME = os.getenv('DBNAME')
    DBUSER = os.getenv('DBUSER')
    DBPASSWORD = os.getenv('DBPASSWORD')    

    conn = psycopg2.connect(
        host=DBHOST,
        port=DBPORT,
        dbname=DBNAME,
        user=DBUSER,
        password=DBPASSWORD
    )
    
    query = """
                WITH TitleCounts AS (
                    SELECT title, COUNT(title) AS no_title 
                    FROM health_video  
                    GROUP BY title 
                    HAVING COUNT(title) = 3
                ),
                RankedVideos AS (
                    SELECT title, link,
                        ROW_NUMBER() OVER (PARTITION BY title ORDER BY link) AS rn
                    FROM health_video
                    WHERE title IN (SELECT title FROM TitleCounts)
                )
                SELECT link, title
                FROM RankedVideos
                WHERE rn = 1
                ORDER BY title ASC;
    """
    
    cur = conn.cursor()
    cur.execute(query)

    rows = cur.fetchall()
    conn.close()
    return rows

# Step 2: Download videos from YouTube with highest quality
def download_videos(rows):
    video_paths = []  # List to store video paths
    for row in rows[:]:
        link = row[0]  # Access link from the dictionary
        yt = YouTube(link)
        stream = yt.streams.get_highest_resolution()
        video_path = stream.download(output_path='downloaded_videos/set1/')
        # Add downloaded video path to the list for further processing
        video_paths.append(video_path)

    return video_paths  # Return the list of video paths for further processing

def download_videos_1080(rows):
    video_paths = []  # List to store video paths
    for row in rows[:]:
        link = row[0]  # Access link from the dictionary
        yt = YouTube(link)

        video_stream = yt.streams.filter(res="1080p", progressive=False, file_extension="mp4").first()
        audio_stream = yt.streams.filter(only_audio=True).first()

        if video_stream and audio_stream:
            video_path = video_stream.download(output_path='downloaded_videos/set2/', filename_prefix="video_")
            audio_path = audio_stream.download(output_path='downloaded_videos/set2/', filename_prefix="audio_")

            video = VideoFileClip(video_path)
            audio = AudioFileClip(audio_path)

            final_clip = video.set_audio(audio)
            final_path = f"downloaded_videos/set2/{yt.title}.mp4"
            final_clip.write_videofile(final_path)
            
            # Delete the separate video and audio files
            os.remove(video_path)
            os.remove(audio_path)

            video_paths.append(final_path)

    return video_paths


# Step 3: Edit and create a new video based on cuts
def edit_video(video_path, start, end, savetitle):
    video = mpy.VideoFileClip(video_path)
    clip = video.subclip(start, end)
    clip.write_videofile(savetitle, threads=4, fps=24,
                         codec="libx264",
                         preset="slow",
                         ffmpeg_params=["-crf", "24"])
    video.close()
    
from moviepy.editor import VideoFileClip, concatenate_videoclips

def time_to_seconds(time_str):
    # Converts time in 'hh:mm:ss.xx' format to seconds
    h, m, s = map(float, time_str.split(':'))
    return h * 3600 + m * 60 + s


# Download videos

In [5]:
rows = get_youtube_links()
video_paths = download_videos_1080(rows)
print(video_paths)

RegexMatchError: regex_search: could not find match for (?:v=|\/)([0-9A-Za-z_-]{11}).*

# connect to DB

In [59]:
DBHOST = os.getenv('DBHOST')
DBPORT = os.getenv('DBPORT')
DBNAME = os.getenv('DBNAME')
DBUSER = os.getenv('DBUSER')
DBPASSWORD = os.getenv('DBPASSWORD')    

conn = psycopg2.connect(
    host=DBHOST,
    port=DBPORT,
    dbname=DBNAME,
    user=DBUSER,
    password=DBPASSWORD
)


# update database 
## init function

In [60]:
import os
from dotenv import load_dotenv
from googleapiclient.discovery import build
import re

# Load environment variables from .env file
load_dotenv()

# Get API key and channel ID from environment variables
api_key = os.getenv('YT_API_KEY')
channel_id = os.getenv('YT_CHANNEL_ID')

# Build the YouTube Data API service
youtube = build('youtube', 'v3', developerKey=api_key)

# Establish a connection to the PostgreSQL database
cursor = conn.cursor()

def get_video_details():
    try:
        # Get the playlist items for the uploads playlist of your channel
        playlist_response = youtube.channels().list(
            part='contentDetails',
            id=channel_id
        ).execute()

        uploads_playlist_id = playlist_response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

        # Initialize variables for pagination
        next_page_token = None
        video_details = []

        # Specify the desired count
        desired_count = 14
        current_count = 0

        while True:
            # Retrieve playlist items with pagination
            playlist_items_response = youtube.playlistItems().list(
                part='contentDetails',
                playlistId=uploads_playlist_id,
                pageToken=next_page_token
            ).execute()

            # Extract video IDs from playlist items
            video_ids = [item['contentDetails']['videoId'] for item in playlist_items_response['items']]

            # Retrieve details for each video
            for video_id in video_ids:
                video_name = get_video_name(video_id)
                video_link = f'https://youtu.be/{video_id}'
                title, reps = split_title(video_name)

                # Insert data into the database
                insert_query = """
                        INSERT INTO public.health_video
                        (title, link, reps, tag, order_number, is_active, duration, "level", exercise_id, sport_equipment, important_angles, coach, health_topic_id, thumb_img, std_pose, preview_link, pose_status, link_video_id, pose, exercise_group_id)
                        VALUES
                        (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """

                # Assuming 'null' is the Python None value
                values = (
                    title,  # title
                    video_link,  # video_link
                    reps,  # reps
                    '#',  # tag
                    None,  # order_number
                    True,  # is_active
                    None,  # duration
                    None,  # level
                    None,  # exercise_id
                    None,  # sport_equipment
                    '{"important": [], "normal": []}',  # important_angles
                    'Quang',  # coach
                    None,  # health_topic_id
                    None,  # thumb_img
                    '{}',  # std_pose
                    None,  # preview_link
                    '{}',  # pose_status
                    None,  # link_video_id
                    None,  # pose
                    None  # exercise_group_id
                )


                cursor.execute(insert_query, values)
                conn.commit()

                video_details.append({'name': title, 'link': video_link, 'reps': reps})

                # Increment the count and check if the desired count is reached
                current_count += 1
                if current_count >= desired_count:
                    break

            # Check for the presence of next page token
            next_page_token = playlist_items_response.get('nextPageToken')

            # Exit the loop if there is no next page or the desired count is reached
            if not next_page_token or current_count >= desired_count:
                break

        return video_details

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def get_video_name(video_id):
    try:
        # Retrieve details for the video
        video_response = youtube.videos().list(
            part='snippet',
            id=video_id
        ).execute()

        # Extract video name
        video_name = video_response['items'][0]['snippet']['title']
        return video_name

    except Exception as e:
        print(f"An error occurred while getting video name: {e}")
        return None

def split_title(video_name):
    # Use regex to extract exercise name and reps
    match = re.match(r"^(.+?)(\s*(?:Set|S)?\s*\d+)$", video_name)
    if match:
        title = match.group(1).strip()  # Extracting exercise name
        reps = match.group(2).strip()   # Extracting reps
        return title, reps
    else:
        print("Unable to split title and reps.")
        return None, None

# def split_title(video_name):
#     # Use regex to extract exercise name and reps
#     match = re.match(r"^(.*?)(?:\s+(?:Set|S)?\d+)?\s*([A-Z])?\s*(\d+)?$", video_name)
#     if match:
#         # Extracting exercise name
#         title = match.group(1).strip()
#         # Remove "Set 1" or "S1" pattern from the title
#         title = re.sub(r"\b(?:Set|S)\d+\b", "", title).strip()
#         # Extracting letter
#         letter = match.group(2) or ""  # If no letter, return empty string
#         # Extracting reps
#         reps = match.group(3) or ""  # If no reps, return empty string
#         return title, letter, reps
#     else:
#         print("Unable to split title, letter and reps.")
#         return None, None, None


def extract_video_id(video_link):
    # Use regex to extract video ID from the YouTube link
    match = re.match(r'^https://youtu\.be/([A-Za-z0-9_-]+)$', video_link)
    if match:
        return match.group(1)
    else:
        print("Invalid YouTube link format")
        return None

def get_video_info():
    try:
        # Execute the SQL query to retrieve video links from the database
        cursor.execute("SELECT link FROM health_video")
        video_links = cursor.fetchall()

        video_details = []

        # Iterate through each video link
        for video_link in video_links:
            video_id = extract_video_id(video_link[0])
            if video_id:
                video_name = get_video_name(video_id)
                if video_name:
                    title, reps = split_title(video_name)
                    if title:
                        title = title.title()  # Capitalize the first letter
                    video_details.append({'link': video_link[0], 'title': title, 'reps': reps})

        return video_details

    except Exception as e:
        print(f"An error occurred: {e}")
        return None



# insert new video into DB

In [ ]:

if __name__ == "__main__":
    details = get_video_details()
    if details:
        for video in details:
            print(f"Name: {video['name']}, Link: {video['link']}, Reps: {video['reps']}")


# get title and reps from youtube link

In [62]:
def get_video_name(video_id, video_link):
    try:
        # Retrieve details for the video
        video_response = youtube.videos().list(
            part='snippet',
            id=video_id
        ).execute()

        # Check if the response contains items
        if 'items' in video_response and video_response['items']:
            # Extract video name
            video_name = video_response['items'][0]['snippet']['title']
            return video_name
        else:
            print(f"No video details found for video link: {video_link}")
            return None

    except Exception as e:
        print(f"An error occurred while getting video name: {e} for video link: {video_link}")
        return None
    
def get_video_info():
    # Execute the SQL query to retrieve video links from the database
    cursor.execute("SELECT link FROM health_video ")
    video_links = cursor.fetchall()

    video_details = []

    # Iterate through each video link
    for video_link in video_links:
        video_id = extract_video_id(video_link[0])
        if video_id:
            video_name = get_video_name(video_id, video_link)  # Pass the video link to get_video_name
            if video_name:
                title,  reps = split_title(video_name)
                if title:
                    original_title = title.title()  # Capitalize the first letter
                    # Split the title into words and replace only if the word is exactly "Set" or "S"
                    title_words = original_title.split()
                    title_words = [word if word.lower() not in {"set", "s"} else "" for word in title_words]
                    title = " ".join(title_words)
                    if reps.isdigit():  # Check if reps is a valid integer
                        reps = int(reps)
                    else:
                        reps = 0
                    # Update the health_video table with the modified title and reps
                    cursor.execute("UPDATE health_video SET title = %s, reps = %s WHERE link = %s", (title, reps, video_link[0]))
                    conn.commit()  # Commit the transaction
                video_details.append({'link': video_link[0], 'title': original_title, 'reps': reps})

    return video_details

if __name__ == "__main__":
    details = get_video_info()
    if details:
        for video in details:
            print(f"Title: {video['title']}, Link: {video['link']}, Reps: {video['reps']}")


Title: Lateral Raise, Link: https://youtu.be/vfLjoeST7dw, Reps: 24
Title: Burpee F, Link: https://youtu.be/LjpO57in6vk, Reps: 0
Title: Burpee Set, Link: https://youtu.be/W7SNKODa9Iw, Reps: 2
Title: Bulgarian Split Squat Set, Link: https://youtu.be/tlnUVpYSm3c, Reps: 3
Title: Calf Raise, Link: https://youtu.be/B-P5TkWrM6A, Reps: 30
Title: Crunch, Link: https://youtu.be/MG2GYmoekdI, Reps: 15
Title: Triceps Extension, Link: https://youtu.be/B3JJWPIoSAw, Reps: 26
Title: Triceps Extension, Link: https://youtu.be/Dth-4pwDIio, Reps: 14
Title: Dumbell Squat, Link: https://youtu.be/yFOpbryQrYY, Reps: 12
Title: Dumbell Calf Curl, Link: https://youtu.be/MdPM2NAF35Q, Reps: 15
Title: Dumbell Lunge, Link: https://youtu.be/D4M7gD2NmpE, Reps: 24
Title: Squat Jump, Link: https://youtu.be/LrN1gXAlEb8, Reps: 12
Title: Laying Legs Raise, Link: https://youtu.be/UV18lsnqsBU, Reps: 12
Title: Shoulder Press, Link: https://youtu.be/ZzJEXhWsWew, Reps: 0
Title: Romanian Deadlift, Link: https://youtu.be/OUSE3TwyE

# delete invalid links

In [25]:
# List of video links to delete
video_links_to_delete = [
    'https://youtu.be/vC4vKw5wFQU',
    'https://youtu.be/NtNHdPWLEkA',
    'https://youtu.be/5bLLKbudhkk',
    'https://youtu.be/vFMJ2NVocr0',
    'https://youtu.be/c7mJZb-gbUA'
]

try:
    # Delete records with specified video links
    for video_link in video_links_to_delete:
        cursor.execute("DELETE FROM health_video WHERE link = %s", (video_link,))
        conn.commit()
        print(f"Deleted record with video link: {video_link}")

except Exception as e:
    print(f"An error occurred while deleting records: {e}")

# Close the cursor and connection
cursor.close()
conn.close()


Deleted record with video link: https://youtu.be/vC4vKw5wFQU
Deleted record with video link: https://youtu.be/NtNHdPWLEkA
Deleted record with video link: https://youtu.be/5bLLKbudhkk
Deleted record with video link: https://youtu.be/vFMJ2NVocr0
Deleted record with video link: https://youtu.be/c7mJZb-gbUA


# close cursor

In [ ]:

# Close the database connection after use
cursor.close()
conn.close()

# Add segments with timer

In [ ]:
import json

from moviepy.video.VideoClip import ColorClip
from moviepy.video.compositing.CompositeVideoClip import CompositeVideoClip

def add_segment_and_timer(original_video_path, background_cover, timer_clip_path, start_time_original, start_time_segment, start_timer, background_clip_cut, output_video_path="result_video.mp4"):
    # Load the original video
    original_video = VideoFileClip(original_video_path)

    # Convert time strings to seconds
    start_time_original_sec = time_to_seconds(start_time_original)
    start_time_segment_sec = (time_to_seconds(start_time_segment[0]), time_to_seconds(start_time_segment[1]))
    
    # Extract the segment from the original video based on start and end times
    segment_clip = original_video.subclip(start_time_segment_sec[0], start_time_segment_sec[1])
    
    # Calculate duration of the segment
    segment_duration = segment_clip.duration
    
    # Find the start and end times for insertion in the original video
    start_time_insertion = start_time_original_sec 
    # start_time_insertion = start_time_original_sec + segment_duration
    
    # Extract the segments before and after the insertion point in the original video
    before_segment = original_video.subclip(0, start_time_insertion)
    after_segment = original_video.subclip(start_time_insertion, original_video.duration)
    
    # Concatenate the segments (before, inserted segment, after) to create the final video
    final_video = concatenate_videoclips([before_segment, segment_clip, after_segment])
    
    
    # Load the timer clip
    timer_clip = VideoFileClip(timer_clip_path)
    
    start_timer_sec = time_to_seconds(start_timer)

    # Create a white box clip
    # white_box = ColorClip((210, 210), col=(255, 255, 255)).set_duration(start_timer_sec - time_to_seconds('00:00:04.37')).set_start(time_to_seconds('00:00:04.37'))
    subclip_from_5_sec = final_video.subclip(time_to_seconds(background_cover))
    subclip_from_5_sec_width = subclip_from_5_sec.w
    print(subclip_from_5_sec_width)

    # Create a background clip
    if background_clip_cut == "left":
        background_clip = subclip_from_5_sec.crop(x1=250, y1=200, width=600, height=subclip_from_5_sec.h // 5).set_duration(start_timer_sec - time_to_seconds(background_cover)).set_start(time_to_seconds(background_cover))
    elif background_clip_cut == "right":
        background_clip = subclip_from_5_sec.crop(x1=subclip_from_5_sec_width - 250, y1=200, width=600, height=subclip_from_5_sec.h // 5).set_duration(start_timer_sec - time_to_seconds(background_cover)).set_start(time_to_seconds(background_cover))
        
    # Resize the cropped area to a width of 600 pixels
    background_clip = background_clip.resize((620, 320))
    
    # Add the white box to the final video
    final_video = CompositeVideoClip([final_video, background_clip])

    # Cut the video into 2 parts start from the start_timer
    video_clip_cut = final_video.subclip(start_timer_sec, final_video.duration)
    
    # set mute
    video_clip_cut = video_clip_cut.set_audio(None)
    
    # Cut the timer clip
    end_timer_subclip_sec = timer_clip.duration  
    video_clip_cut_duration = video_clip_cut.duration
    
    timer_clip_cut = timer_clip.subclip(end_timer_subclip_sec - video_clip_cut_duration, end_timer_subclip_sec)
    
    # Apply fade-in effect to the timer clip
    timer_clip_cut = timer_clip_cut.fadein(1)
    
    # Resize and position the cut timer clip
    timer_clip_cut_resized = timer_clip_cut.resize(height=final_video.h // 5)
    timer_clip_cut_positioned = timer_clip_cut_resized.set_position((35, 35))
    
    # Add the cut timer clip to the final video
    video_clip_cut_with_timer = CompositeVideoClip([video_clip_cut, timer_clip_cut_positioned])
    
    final_video_with_timer = concatenate_videoclips([final_video.subclip(0, start_timer_sec), video_clip_cut_with_timer])
    
    # Load the replacement clip (5-secs.mp4)
    replacement_clip = VideoFileClip('D:/0_Materials/0_timer/IX-HEALTH LOGO.mp4')

    # Add a fade-out effect to the replacement clip
    replacement_clip_faded = replacement_clip.fadeout(2)  # Adjust the duration of the fade-out as needed
    
    # Get the duration of the replacement clip
    replacement_duration = replacement_clip.duration
    
    # Cut the final video from the beginning with the duration of the replacement clip
    final_video_cut = final_video_with_timer.subclip(replacement_duration)

    # Concatenate the replacement clip at the beginning
    final_video_with_replacement = concatenate_videoclips([replacement_clip_faded, final_video_cut])

    # Mute the video
    # final_video_with_timer = final_video_with_timer.set_audio(None)
    
    # Write the final video to a file
    final_video_with_replacement.write_videofile(output_video_path, codec="libx264", fps=24)

    final_video_with_replacement.close()
    
with open('video_info.json') as f:
    data = json.load(f)
    # d = data[1]
    for d in data[0:1]:
        for segment in d['time_segment']:
        # segment = d['time_segment'][2]
            add_segment_and_timer(d['original_video_path'],
                                d['background_cover'],
                                d['timer_clip_path'], 
                                d['start_time_original'], 
                                segment['start_time_segment'],
                                d['start_timer'], 
                                d['background_clip_cut'],
                                f'edited_videos/{segment["output_video_path"]}'
                                )
            print(f"Done: {segment['output_video_path']}")

# add logo

In [ ]:
from moviepy.editor import VideoFileClip, CompositeVideoClip
from moviepy.video.fx.fadein import fadein
import json

def add_logo(original_video_path, output_video_path):
    # Load the original video
    original_video = VideoFileClip(original_video_path)

    # Load the logo clip
    logo_clip_path = 'D:/0_Materials/0_timer/IX-HEALTH LOGO.mp4'
    logo_clip = VideoFileClip(logo_clip_path)

    # Add a fade-in effect to the logo clip
    logo_clip = fadein(logo_clip, duration=2)

    # Position the logo clip on the bottom right corner
    logo_position = (original_video.size[0] - logo_clip.size[0] - 10, original_video.size[1] - logo_clip.size[1] - 10)
    logo_clip = logo_clip.set_position(logo_position)

    # Composite the logo clip onto the original video
    final_video = CompositeVideoClip([original_video, logo_clip])

    # Write the final video to a file
    final_video.write_videofile(output_video_path, codec="libx264", fps=24)
    final_video.close()

with open('video_info.json') as f:
    data = json.load(f)
    for d in data[0:1]:
        # for segment in d['time_segment']:
        segment = d['time_segment'][0]
        add_logo(d['original_video_path'], f'edited_videos/{segment["output_video_path"]}')
        print(f"Done: {segment['output_video_path']}")
